In [1]:
from fmm import Network,NetworkGraph,FastMapMatch,FastMapMatchConfig,UBODT,GPSConfig,ResultConfig
import pandas as pd
from shapely.wkt import loads
from shapely.geometry import mapping
import json
from ipyleaflet import Map, GeoJSON, WidgetControl
import ipywidgets as widgets

In [2]:
#load network data and graph 
network = Network("../osmnx_example/beijing/edges.shp","fid", "u", "v")
print("Nodes {} edges {}".format(network.get_node_count(),network.get_edge_count()))
graph = NetworkGraph(network)

[2025-02-25 18:42:05.485] [info] [network.cpp:72] Read network from file ../osmnx_example/beijing/edges.shp
Nodes 33999 edges 85538
[2025-02-25 18:42:06.427] [info] [network.cpp:172] Number of edges 85538 nodes 33999
[2025-02-25 18:42:06.427] [info] [network.cpp:173] Field index: id 18 source 0 target 1
[2025-02-25 18:42:06.515] [info] [network.cpp:176] Read network done.
[2025-02-25 18:42:06.517] [info] [network_graph.cpp:17] Construct graph from network edges start
[2025-02-25 18:42:06.529] [info] [network_graph.cpp:30] Graph nodes 33999 edges 85538
[2025-02-25 18:42:06.529] [info] [network_graph.cpp:31] Construct graph from network edges end


### Precompute an UBODT file

**This step can be skipped if you already created one UBODT file.**

In [3]:
#from fmm import UBODTGenAlgorithm
#ubodt_gen = UBODTGenAlgorithm(network,graph)
#status = ubodt_gen.generate_ubodt("../osmnx_example/beijing/ubodt.txt", 4, binary=False, use_omp=True)
#print(status)

### FMM model creation and map matching

match single trajectory

In [ ]:
df = pd.read_csv("../osmnx_example/beijing/trajectories.csv", sep=";")

#trajectory 1
wkt = df.loc[df["id"] == 71, "geom"].values[0]

print(wkt)

ubodt = UBODT.read_ubodt_csv("../osmnx_example/beijing/ubodt.txt")
model = FastMapMatch(network,graph,ubodt)

# configuration parameters
k = 3
radius = 0.4
gps_error = 0.5
fmm_config = FastMapMatchConfig(k,radius,gps_error)

result = model.match_wkt(wkt,fmm_config)
print("Matched path: ", list(result.cpath))
print("Matched edge for each point: ", list(result.opath))
print("Matched edge index ",list(result.indices))
print("Matched geometry: ",result.mgeom.export_wkt())
print("Matched point ", result.pgeom.export_wkt())

In [ ]:
import folium
from shapely.wkt import loads

original_geom = loads(wkt)
matched_geom = loads(result.mgeom.export_wkt())

m = folium.Map(location=[original_geom.coords[0][1], original_geom.coords[0][0]], zoom_start=16)

folium.PolyLine([(lat, lon) for lon, lat in original_geom.coords], color="red", weight=3, opacity=0.8, tooltip="Original GPS Trajectory").add_to(m)
folium.PolyLine([(lat, lon) for lon, lat in matched_geom.coords], color="blue", weight=3, opacity=0.8, tooltip="Matched Path").add_to(m)

for lon, lat in original_geom.coords:
    folium.CircleMarker(location=[lat, lon], radius=3, color="red", fill=True, fill_color="red").add_to(m)
for lon, lat in matched_geom.coords:
    folium.CircleMarker(location=[lat, lon], radius=3, color="blue", fill=True, fill_color="blue").add_to(m)

# Display map 
m

### Map Matching after obfuscation

In [3]:
ubodt = UBODT.read_ubodt_csv("../osmnx_example/beijing/ubodt.txt")
model = FastMapMatch(network,graph,ubodt)

[2025-02-25 18:42:13.787] [info] [ubodt.cpp:208] Reading UBODT file (CSV format) from ../osmnx_example/beijing/ubodt.txt
[2025-02-25 18:42:14.750] [info] [ubodt.cpp:236] Read rows 1000000
[2025-02-25 18:42:15.672] [info] [ubodt.cpp:236] Read rows 2000000
[2025-02-25 18:42:16.651] [info] [ubodt.cpp:236] Read rows 3000000
[2025-02-25 18:42:17.479] [info] [ubodt.cpp:236] Read rows 4000000
[2025-02-25 18:42:18.217] [info] [ubodt.cpp:236] Read rows 5000000
[2025-02-25 18:42:19.040] [info] [ubodt.cpp:236] Read rows 6000000
[2025-02-25 18:42:20.209] [info] [ubodt.cpp:236] Read rows 7000000
[2025-02-25 18:42:21.100] [info] [ubodt.cpp:236] Read rows 8000000
[2025-02-25 18:42:22.259] [info] [ubodt.cpp:236] Read rows 9000000
[2025-02-25 18:42:23.250] [info] [ubodt.cpp:236] Read rows 10000000
[2025-02-25 18:42:23.939] [info] [ubodt.cpp:236] Read rows 11000000
[2025-02-25 18:42:24.738] [info] [ubodt.cpp:236] Read rows 12000000
[2025-02-25 18:42:25.441] [info] [ubodt.cpp:236] Read rows 13000000
[202

In [ ]:
df = pd.read_csv("../osmnx_example/beijing/perturbed_trajectories.csv", sep=";")
df_original = pd.read_csv("../osmnx_example/beijing/trajectories.csv", sep=";")

wkt = df_original.loc[df_original["id"] == 71, "geom"].values[0]

#trajectory 1
wkt_pert = df.loc[df["id"] == 71, "geom"].values[0]

# configuration parameters
k = 8
radius = 300
gps_error = 300
fmm_config = FastMapMatchConfig(k,radius,gps_error)

result_pert = model.match_wkt(wkt_pert,fmm_config)
print("Matched path: ", list(result_pert.cpath))
print("Matched edge for each point: ", list(result_pert.opath))
print("Matched edge index ",list(result_pert.indices))
print("Matched geometry: ",result_pert.mgeom.export_wkt())
print("Matched point ", result_pert.pgeom.export_wkt())

Matched path:  [40395, 40408, 40407, 26910, 5318, 40397, 5325, 17349, 5310, 64918, 64897, 16913, 1266, 17355, 63087, 17354, 78517, 17350, 5323, 40396, 5317, 26911, 40405, 40409, 40395, 40408, 40407, 26910, 5318, 40397, 5326, 5321, 17339, 1273, 13964, 580, 17344, 64905, 23037, 55856, 793, 3945, 3951, 28032, 28036, 28040, 80476, 28044, 3949, 56516, 56522, 80458, 80455, 80460, 80461, 17337, 80491, 64938, 17344, 64905, 23037, 55856, 793, 3945, 3951, 28032, 28036, 28040, 80476, 28044, 3950, 3954, 27591, 12607, 12240, 12442, 12589, 3917, 71927, 71919, 71921, 71937, 71922, 71917, 71928, 71930, 71964, 71956, 71959, 12268, 65104, 12269, 12132, 12135, 12151, 12155, 27639, 12298, 68193, 68189, 64772, 12245, 68183, 75406, 68185, 75410, 58988, 12436, 34835, 69656, 69652, 57638, 69653, 58964, 58965, 48486, 27729, 63792, 57596, 75454, 75462, 75465, 12232, 15901, 57624, 27652, 72284, 70985, 71699, 4216, 71695, 60444, 71652, 60441, 71636, 16388, 16366, 16360, 16349, 16343, 60460, 16329, 21883, 16327, 1

In [7]:
import folium
from shapely.wkt import loads

original_geom = loads(wkt)
pert_geom = loads(wkt_pert)
pert_matched_geom = loads(result_pert.mgeom.export_wkt())

m = folium.Map(location=[original_geom.coords[0][1], original_geom.coords[0][0]], zoom_start=16)

folium.PolyLine([(lat, lon) for lon, lat in original_geom.coords], color="red", weight=3, opacity=0.8, tooltip="Original GPS Trajectory").add_to(m)
folium.PolyLine([(lat, lon) for lon, lat in pert_geom.coords], color="blue", weight=3, opacity=0.8, tooltip="Perturbed GPS Trajectory").add_to(m)
folium.PolyLine([(lat, lon) for lon, lat in pert_matched_geom.coords], color="green", weight=3, opacity=0.8, tooltip="Matched Path").add_to(m)

for lon, lat in original_geom.coords:
    folium.CircleMarker(location=[lat, lon], radius=3, color="red", fill=True, fill_color="red").add_to(m)
for lon, lat in pert_geom.coords:
    folium.CircleMarker(location=[lat, lon], radius=3, color="blue", fill=True, fill_color="blue").add_to(m)
for lon, lat in pert_matched_geom.coords:
    folium.CircleMarker(location=[lat, lon], radius=3, color="green", fill=True, fill_color="green").add_to(m)
# Display map 
m